<a href="https://colab.research.google.com/github/jiin124/2021_2_SAI_ML/blob/main/11%EC%A3%BC%EC%B0%A8/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

os.environ['KAGGLE_USERNAME']='jiinpp'
os.environ['KAGGLE_KEY']='b06d525a9f66198051ccb648b091778a'

!kaggle competitions download -c word2vec-nlp-tutorial

 65% 17.0M/26.0M [00:00<00:00, 44.5MB/s]
100% 26.0M/26.0M [00:00<00:00, 73.6MB/s]
  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 82.0MB/s]
 40% 5.00M/12.6M [00:00<00:00, 32.0MB/s]
100% 12.6M/12.6M [00:00<00:00, 61.4MB/s]
 39% 5.00M/13.0M [00:00<00:00, 36.1MB/s]
100% 13.0M/13.0M [00:00<00:00, 63.3MB/s]


In [3]:
!unzip '*.zip'

Archive:  unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  

Archive:  labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    

Archive:  testData.tsv.zip
  inflating: testData.tsv            

3 archives were successfully processed.


In [5]:
import pandas as pd

df=pd.read_csv("labeledTrainData.tsv",header=0,sep="\t",quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [6]:
print(df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [7]:
import re

#<br> html 태그는 replace함수로 공백으로 전환
df['review']=df['review'].str.replace('<br />',' ')

#파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 전환
df['review']=df['review'].apply(lambda x:re.sub("[^a-zA-Z]"," ",x))

In [8]:
from sklearn.model_selection import train_test_split

class_df=df['sentiment']
feature_df=df.drop(['id','sentiment'],axis=1,inplace=False)

In [9]:
X_train,X_test,y_train,y_test=train_test_split(feature_df,class_df,test_size=0.3,
                                               random_state=156)

In [10]:
X_train.shape,X_test.shape

((17500, 1), (7500, 1))

In [28]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score


#스톱워드를 english,filtering,ngram은 (1,2)로 설정해 countVectorization수행
#LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))
    ])

#Pipeline 객체를 이용해 fit(),predict()로 학습/에측 수행. predict_proba()는 roc_auc 때문에 수행
pipeline.fit(X_train['review'],y_train)
pred=pipeline.predict(X_test['review'])
pred_probs=pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


예측 정확도는 0.8860, ROC-AUC는 0.9503


In [29]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


#스톱워드 english,filtering,ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행
#LogisticRegression의 C는 10으로 설정

pipeline=Pipeline([
                   ('tfidf_vect', TfidfVectorizer(stop_words='english',ngram_range=(1,2))),
                   ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train['review'],y_train)
pred=pipeline.predict(X_test['review'])
pred_probs=pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test,pred),
                                                 roc_auc_score(y_test,pred_probs)))

예측 정확도는 0.8936, ROC-AUC는 0.9598


## 비지도 기반 감성 분석 소개

In [30]:
import nltk
nltk.download('all')



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [31]:
from nltk.corpus import wordnet as wn

term='present'

#'present'라는 단어로 wordnet의 synsets생성.
synsets=wn.synsets(term)
print('synsets() 반환 thpe:',type(synsets))
print('synsets() 반환 값 개수:',len(synsets))
print('synsets() 반환 값:',synsets)

synsets() 반환 thpe: <class 'list'>
synsets() 반환 값 개수: 18
synsets() 반환 값: [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [32]:
for synset in synsets:
  print("#### Synset name: ",synset.name(),'####')
  print('POS :',synset.lexname())
  print('Definision:',synset.definition())
  print('Lemmas:',synset.lemma_names())

#### Synset name:  present.n.01 ####
POS : noun.time
Definision: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
#### Synset name:  present.n.02 ####
POS : noun.possession
Definision: something presented as a gift
Lemmas: ['present']
#### Synset name:  present.n.03 ####
POS : noun.communication
Definision: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
#### Synset name:  show.v.01 ####
POS : verb.perception
Definision: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
#### Synset name:  present.v.02 ####
POS : verb.communication
Definision: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
#### Synset name:  stage.v.01 ####
POS : verb.creation
Definision: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 'represent']
#### Synset na

In [33]:
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities=[tree,lion,tiger,cat,dog]
similarities=[]
entity_names=[entity.name().split('.')[0] for entity in entities]

#단어별 synset을 반복하면서 다른 단어의 synset과 유사도를 측정
for entity in entities:
  similarity=[round(entity.path_similarity(compared_entity),2)
  for compared_entity in entities]
  similarities.append(similarity)

#개별 단어별 synset과 다른 단어의 synset과의 유사도를 dataframe 형태로 저장
similarity_df=pd.DataFrame(similarities,columns=entity_names,index=entity_names)
similarity_df



,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [34]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 갯수:', len(senti_synsets))
print('senti_synsets() 반환 값 :', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 갯수: 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [35]:
import nltk
from nltk.corpus import sentiwordnet as swn

father=swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())
print('\n')

fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ',fabulous .pos_score())
print('fabulous 부정감성 지수: ',fabulous .neg_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125


## SentiWordNet을 이용한 영화 감상평 감성 분석

품사 태깅을 수행하는 함수




In [37]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return 



In [38]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue                   

            lemma = lemmatizer.lemmatize(word, pos=wn_tag)

            if not lemma:
                continue

            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wn.synsets(lemma , pos=wn_tag)

            if not synsets:
                continue
                
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1
    
    return 0



In [40]:
df['preds']=df['review'].apply(lambda x: swn_polarity(x))
y_target=df['sentiment'].values
preds=df['preds'].values

In [41]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
from sklearn.metrics import recall_score,f1_score,roc_auc_score
import numpy as np

print(confusion_matrix(y_target,preds))
print("정확도:", accuracy_score(y_target , preds))
print("정밀도:", precision_score(y_target , preds))
print("재현율:", recall_score(y_target, preds))



[[7668 4832]
 [3636 8864]]
정확도: 0.66128
정밀도: 0.647196261682243
재현율: 0.70912
